<a href="https://colab.research.google.com/github/mshahmeer1/Radiological-phenotypes-of-ARDS/blob/main/Data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
from google.colab import files

In [ ]:
#user authentication
auth.authenticate_user()

In [ ]:
# Set up the project ID
# Note that the physionet-data project is for data hosting only.
project_id = 'august-tangent-380700'
os.environ['august-tangent-380700'] = project_id

In [ ]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      configuration={'query': {
          'useLegacySql': False
      }})

In [ ]:
query = ''' with mvp AS
(
with ce_stg1 as
(
  SELECT
      ce.subject_id
    , ce.stay_id
    , ce.charttime
    , CASE
        -- merge o2 flows into a single row
        WHEN itemid IN (223834, 227582, 224691) THEN 223834
      ELSE itemid END AS itemid
    , value
    , valuenum
    , valueuom
    , storetime
  FROM physionet-data.mimiciv_icu.chartevents ce
  WHERE ce.value IS NOT NULL
  AND ce.itemid IN
  (
      223834 -- o2 flow
    , 227582 -- bipap o2 flow
    , 224691 -- Flow Rate (L)
    -- additional o2 flow is its own column
    , 227287 -- additional o2 flow
  )
)
, ce_stg2 AS
(
  select
    ce.subject_id
    , ce.stay_id
    , ce.charttime
    , itemid
    , value
    , valuenum
    , valueuom
    -- retain only 1 row per charttime
    -- prioritizing the last documented value
    -- primarily used to subselect o2 flows
    , ROW_NUMBER() OVER (PARTITION BY subject_id, charttime, itemid ORDER BY storetime DESC) as rn
  FROM ce_stg1 ce
)
, o2 AS
(
    -- The below ITEMID can have multiple entires for charttime/storetime
    -- These are totally valid entries, and should be retained in derived tables.
    --   224181 -- Small Volume Neb Drug #1              | Respiratory             | Text       | chartevents
    -- , 227570 -- Small Volume Neb Drug/Dose #1         | Respiratory             | Text       | chartevents
    -- , 224833 -- SBT Deferred                          | Respiratory             | Text       | chartevents
    -- , 224716 -- SBT Stopped                           | Respiratory             | Text       | chartevents
    -- , 224740 -- RSBI Deferred                         | Respiratory             | Text       | chartevents
    -- , 224829 -- Trach Tube Type                       | Respiratory             | Text       | chartevents
    -- , 226732 -- O2 Delivery Device(s)                 | Respiratory             | Text       | chartevents
    -- , 226873 -- Inspiratory Ratio                     | Respiratory             | Numeric    | chartevents
    -- , 226871 -- Expiratory Ratio                      | Respiratory             | Numeric    | chartevents
    -- maximum of 4 o2 devices on at once
    SELECT
        subject_id
        , stay_id
        , charttime
        , itemid
        , value AS o2_device
    , ROW_NUMBER() OVER (PARTITION BY subject_id, charttime, itemid ORDER BY value) as rn
    FROM physionet-data.mimiciv_icu.chartevents
    WHERE itemid = 226732 
    AND value in ('Trach mask', 'Tracheostomy tube', 'Bipap mask', 'CPAP mask', 'Endotracheal tube', 'T-piece')-- oxygen delivery device(s)
)
, stg AS
(
    select
      COALESCE(ce.subject_id, o2.subject_id) AS subject_id
    , COALESCE(ce.stay_id, o2.stay_id) AS stay_id
    , COALESCE(ce.charttime, o2.charttime) AS charttime
    , COALESCE(ce.itemid, o2.itemid) AS itemid
    , ce.value
    , ce.valuenum
    , o2.o2_device
    , o2.rn
    from ce_stg2 ce
    FULL OUTER JOIN o2
      ON ce.subject_id = o2.subject_id
      AND ce.charttime = o2.charttime
    -- limit to 1 row per subject_id/charttime/itemid from ce_stg2
    WHERE ce.rn = 1
)
SELECT
    subject_id
    , MAX(stay_id) AS stay_id
    , charttime
    , MAX(CASE WHEN itemid = 223834 THEN valuenum ELSE NULL END) AS o2_flow
    , MAX(CASE WHEN itemid = 227287 THEN valuenum ELSE NULL END) AS o2_flow_additional
    -- ensure we retain all o2 devices for the patient
    , MAX(CASE WHEN rn = 1 THEN o2_device ELSE NULL END) AS o2_delivery_device_1
    , MAX(CASE WHEN rn = 2 THEN o2_device ELSE NULL END) AS o2_delivery_device_2
    , MAX(CASE WHEN rn = 3 THEN o2_device ELSE NULL END) AS o2_delivery_device_3
    , MAX(CASE WHEN rn = 4 THEN o2_device ELSE NULL END) AS o2_delivery_device_4
FROM stg
GROUP BY subject_id, charttime

)
SELECT
 *
 FROM mvp
 WHERE o2_delivery_device_1 IS NOT NULL
 OR o2_delivery_device_2 IS NOT NULL
 OR o2_delivery_device_3 IS NOT NULL
 OR o2_delivery_device_4 IS NOT NULL


'''

d= run_query(query)
d.head(5)

,subject_id,stay_id,charttime,o2_flow,o2_flow_additional,o2_delivery_device_1,o2_delivery_device_2,o2_delivery_device_3,o2_delivery_device_4
0,10336855,33336719,2151-06-30 04:00:00,29.9,NaN,Endotracheal tube,None,None,None
1,10398981,34166660,2125-09-26 11:00:00,42.4,NaN,Endotracheal tube,None,None,None
2,10454216,39132344,2164-09-19 11:00:00,41.7,NaN,Tracheostomy tube,None,None,None
3,10472107,32647330,2179-07-10 03:00:00,43.8,NaN,Endotracheal tube,None,None,None
4,10771812,31836611,2174-03-13 04:00:00,45.0,NaN,Endotracheal tube,None,None,None


In [ ]:
d.to_csv('MV.csv')
files.download('MV.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

 SELECT 
   subject_id
    , stay_id
    , charttime
    , value
    , valuenum
    , valueuom
  FROM physionet-data.mimiciv_icu.chartevents 
  WHERE itemid = 220224
ORDER BY subject_id, stay_id, charttime
'''
pO2 = run_query(query)

pO2.head(5)


,subject_id,stay_id,charttime,value,valuenum,valueuom
0,10001884,37510196,2131-01-12 21:04:00,65,65.0,mmHg
1,10001884,37510196,2131-01-13 02:28:00,69,69.0,mmHg
2,10001884,37510196,2131-01-14 07:05:00,91,91.0,mmHg
3,10002013,39060235,2160-05-18 10:26:00,421,421.0,mmHg
4,10002013,39060235,2160-05-18 11:23:00,462,462.0,mmHg


In [ ]:
pO2.to_csv('pO2.csv')
files.download('pO2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT 
      subject_id
    , stay_id
    , charttime
    , value
    , valuenum
    , valueuom,
    CASE
      WHEN valuenum >= 21.0 THEN valuenum * 0.01
      WHEN valuenum >= 1.0 and valuenum < 21.0 THEN (20 + (4 * valuenum)) * 0.01
      WHEN valuenum < 1.0 AND valuenum > 0 THEN valuenum
      WHEN valuenum = 0.0 THEN 0.21
      ELSE NULL
    END as converted_fiO2,


  FROM physionet-data.mimiciv_icu.chartevents
  WHERE itemid = 223835
ORDER BY subject_id, stay_id, charttime
'''
fiO2 = run_query(query)

fiO2.head(5)


,subject_id,stay_id,charttime,value,valuenum,valueuom,converted_fiO2
0,10000980,39765666,2189-06-27 10:00:00,40,40.0,None,0.4
1,10001884,37510196,2131-01-11 04:00:00,100,100.0,None,1.0
2,10001884,37510196,2131-01-11 07:00:00,50,50.0,None,0.5
3,10001884,37510196,2131-01-11 11:00:00,50,50.0,None,0.5
4,10001884,37510196,2131-01-11 12:00:00,50,50.0,None,0.5


In [ ]:
fiO2.to_csv('fiO2.csv')
files.download('fiO2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT * 

FROM physionet-data.mimiciv_icu.procedureevents
  WHERE itemid = 221216

ORDER BY subject_id, stay_id
'''
Xray = run_query(query)

Xray.head(5)


,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,ORIGINALAMOUNT,ORIGINALRATE
0,10002428,28662225,33987268,47220,2156-04-13 09:32:00,2156-04-13 09:33:00,2156-04-13 10:32:00,221216,1.0,None,...,4778858,4778858,Imaging,Task,48.0,0,0,FinishedRunning,1.0,0.0
1,10002428,23473524,35479615,19728,2156-05-11 17:52:00,2156-05-11 17:53:00,2156-05-11 17:52:00,221216,1.0,None,...,2631593,2631593,Imaging,Task,48.4,0,0,FinishedRunning,1.0,0.0
2,10003400,20214994,32128372,32221,2137-02-26 14:54:00,2137-02-26 14:55:00,2137-02-26 14:54:00,221216,1.0,None,...,703087,703087,Imaging,Task,93.0,0,0,FinishedRunning,1.0,0.0
3,10004401,29988601,32773003,65678,2144-01-27 06:35:00,2144-01-27 06:36:00,2144-01-27 06:35:00,221216,1.0,None,...,258963,258963,Imaging,Task,76.0,0,0,FinishedRunning,1.0,0.0
4,10004401,22869003,38292466,16647,2144-04-08 13:41:00,2144-04-08 13:42:00,2144-04-08 13:41:00,221216,1.0,None,...,8609690,8609690,Imaging,Task,77.5,0,0,FinishedRunning,1.0,0.0


In [ ]:
Xray.to_csv('Xray.csv')
files.download('Xray.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT *

FROM `physionet-data.mimiciv_derived.age`

WHERE age >= 18
ORDER BY subject_id, hadm_id
'''
age = run_query(query)

age.head(5)

,subject_id,hadm_id,admittime,anchor_age,anchor_year,age
0,10000032,22595853,2180-05-06 22:23:00,52,2180,52
1,10000032,22841357,2180-06-26 18:27:00,52,2180,52
2,10000032,25742920,2180-08-05 23:44:00,52,2180,52
3,10000032,29079034,2180-07-23 12:35:00,52,2180,52
4,10000068,25022803,2160-03-03 23:16:00,19,2160,19


In [ ]:
age.to_csv('age.csv')
files.download('age.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT 
  dicom
, PatientID
, StudyID
, StudyDate
, StudyTime 
FROM `physionet-data.mimic_cxr.dicom_metadata_string` 
ORDER BY PatientID, StudyDate

'''
mimic_cxr = run_query(query)

mimic_cxr.head(5)

,dicom,PatientID,StudyID,StudyDate,StudyTime
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,21800506,213014.531
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,21800506,213014.531
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,21800626,165500.312
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,21800626,165500.312
4,fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818,10000032,53911762,21800723,080556.875


In [ ]:
mimic_cxr.to_csv('mimic_cxr.csv')
files.download('mimic_cxr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = '''
with mv as (
SELECT distinct stay_id FROM `physionet-data.mimiciv_derived.ventilation` 
where ventilation_status='InvasiveVent'
)

select subject_id, hadm_id, ics.stay_id
FROM `physionet-data.mimiciv_icu.icustays` ics, mv
where ics.stay_id=mv.stay_id

ORDER BY subject_id, ics.stay_id
'''
derived_MV = run_query(query)
derived_MV.head()

<bound method NDFrame.head of        subject_id   hadm_id   stay_id
0        10001884  26184834  37510196
1        10002013  23581541  39060235
2        10002428  23473524  35479615
3        10002428  28662225  38875437
4        10002760  28094813  31831386
...           ...       ...       ...
26887    19999068  21606769  30143796
26888    19999287  20175828  35165301
26889    19999442  26785317  32336619
26890    19999840  21033226  38978960
26891    19999987  23865745  36195440

[26892 rows x 3 columns]>

In [ ]:
derived_MV.to_csv('derived_MV.csv')
files.download('derived_MV.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
with ic as
(select subject_id, hadm_id, stay_id
FROM `physionet-data.mimiciv_icu.icustays`)

SELECT pO2.subject_id, po2, charttime
FROM `physionet-data.mimiciv_derived.bg` pO2, ic
WHERE SPECIMEN= 'ART.' AND ic.subject_id = pO2.subject_id
ORDER BY subject_id
'''
derived_pO2 = run_query(query)
derived_pO2.head()

KeyboardInterrupt: ignored

In [ ]:
derived_pO2.to_csv('derived_pO2.csv')
files.download('derived_pO2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT subject_id, fio2_chartevents, charttime,
    CASE
      WHEN fio2_chartevents >= 21.0 THEN fio2_chartevents * 0.01
      WHEN fio2_chartevents >= 1.0 and fio2_chartevents < 21.0 THEN (20 + (4 * fio2_chartevents)) * 0.01
      WHEN fio2_chartevents < 1.0 AND fio2_chartevents > 0 THEN fio2_chartevents
      WHEN fio2_chartevents = 0.0 THEN 0.21
      ELSE NULL
    END as converted_fiO2,
FROM `physionet-data.mimiciv_derived.bg` 
WHERE SPECIMEN= 'ART.'
ORDER BY subject_id
'''
derived_fiO2 = run_query(query)
derived_fiO2.head()

,subject_id,fio2_chartevents,charttime,converted_fiO2
0,10000935,NaN,2187-10-22 15:40:00,NaN
1,10001884,NaN,2131-01-10 13:15:00,NaN
2,10001884,40.0,2131-01-12 21:04:00,0.4
3,10001884,40.0,2131-01-13 02:28:00,0.4
4,10001884,NaN,2131-01-11 03:42:00,NaN


In [ ]:
derived_fiO2.to_csv('derived_fiO2.csv')
files.download('derived_fiO2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

 SELECT 
   subject_id
   , hadm_id
    , stay_id
    , los
  FROM physionet-data.mimiciv_icu.icustays 
ORDER BY subject_id, hadm_id, stay_id
'''
icu_los = run_query(query)

icu_los.head(5)

,subject_id,hadm_id,stay_id,los
0,10000032,29079034,39553978,0.410266
1,10000980,26913865,39765666,0.497535
2,10001217,24597018,37067082,1.118032
3,10001217,27703517,34592300,0.948113
4,10001725,25563031,31205490,1.338588


In [ ]:
icu_los.to_csv('icu_los.csv')
files.download('icu_los.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

 SELECT 
   subject_id
   , hadm_id

    , hospital_expire_flag
  FROM physionet-data.mimiciv_hosp.admissions
ORDER BY subject_id, hadm_id
'''
hosp_mort = run_query(query)

hosp_mort.head(5)

,subject_id,hadm_id,hospital_expire_flag
0,10000032,22595853,0
1,10000032,22841357,0
2,10000032,25742920,0
3,10000032,29079034,0
4,10000068,25022803,0


In [ ]:
hosp_mort.to_csv('hosp_mort.csv')
files.download('hosp_mort.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = '''
with height as (
SELECT distinct stay_id, height FROM `physionet-data.mimiciv_derived.height` 
)

select subject_id, hadm_id, ics.stay_id
FROM `physionet-data.mimiciv_icu.icustays` ics, mv
where ics.stay_id=mv.stay_id

ORDER BY subject_id, ics.stay_id
'''
derived_MV = run_query(query)
derived_MV.head()

In [ ]:
query = ''' 
with ethnicity as(
  SELECT 
  subject_id,
  race,
  FROM physionet-data.mimiciv_hosp.admissions
  )

SELECT
pts.subject_id, pts.gender,
ethnicity.race
FROM physionet-data.mimiciv_hosp.patients pts
INNER JOIN ethnicity
ON ethnicity.subject_id = pts.subject_id
'''
bmi = run_query(query)

bmi.head(5)

,subject_id,gender,race
0,10686175,M,AMERICAN INDIAN/ALASKA NATIVE
1,11776346,F,ASIAN - ASIAN INDIAN
2,10052990,M,MULTIPLE RACE/ETHNICITY
3,11919492,F,AMERICAN INDIAN/ALASKA NATIVE
4,17408195,M,MULTIPLE RACE/ETHNICITY


In [ ]:
gender_ethnicity = bmi
gender_ethnicity.to_csv('gender_ethnicity.csv')
files.download('gender_ethnicity.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   chartdate,
   result_value, 

  FROM physionet-data.mimiciv_hosp.omr
  WHERE result_name = 'BMI (kg/m2)'
'''
bmi = run_query(query)

bmi.head(5)

,subject_id,chartdate,result_value
0,10030863,2196-03-02,32.7
1,10055729,2142-01-07,33.8
2,10498557,2193-08-03,41.1
3,10591385,2170-09-21,33.2
4,10595272,2171-07-28,42.9


In [ ]:
bmi.to_csv('bmi.csv')
files.download('bmi.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   hadm_id,
   stay_id, 
   intime, outtime,
   los

  FROM physionet-data.mimiciv_icu.icustays
  ORDER BY subject_id, hadm_id, stay_id, intime
'''
stay_id = run_query(query)

stay_id.head(5)

,subject_id,hadm_id,stay_id,intime,outtime,los
0,10000032,29079034,39553978,2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588


In [ ]:
stay_id.to_csv('stay_id.csv')
files.download('stay_id.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 
 SELECT 
   subject_id, 
   stay_id, 
   charttime,
   respiratory_rate_total,
   minute_volume,
   tidal_volume_observed,
   plateau_pressure,
   peep,
   fio2,
   ventilator_mode

  FROM physionet-data.mimiciv_derived.ventilator_setting
  ORDER BY subject_id, stay_id, charttime
'''
physiology = run_query(query)

physiology.head(5)

,subject_id,stay_id,charttime,respiratory_rate_total,minute_volume,tidal_volume_observed,plateau_pressure,peep,fio2,ventilator_mode
0,10000980,39765666,2189-06-27 09:00:00,27.0,11.2,393.0,NaN,5.0,NaN,CPAP/PSV
1,10000980,39765666,2189-06-27 10:00:00,NaN,NaN,NaN,NaN,NaN,40.0,Standby
2,10001884,37510196,2131-01-11 04:00:00,20.0,9.0,479.0,24.0,5.0,100.0,None
3,10001884,37510196,2131-01-11 07:00:00,20.0,8.0,408.0,22.0,5.1,50.0,None
4,10001884,37510196,2131-01-11 11:00:00,20.0,8.3,430.0,23.0,5.2,50.0,None


In [ ]:
physiology.to_csv('physiology.csv')
files.download('physiology.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

SELECT
oasis.subject_id, 
oasis.hadm_id,
oasis.stay_id,
oasis.oasis,
oasis.oasis_prob,

FROM physionet-data.mimiciv_derived.oasis oasis
'''
oasis_score = run_query(query)

oasis_score.head(5)

,subject_id,hadm_id,stay_id,oasis,oasis_prob
0,13207437,22130065,38663369,11,0.008392
1,19853481,27529179,36682195,13,0.010803
2,19657612,26264002,31552189,13,0.010803
3,15175657,24212233,30725529,13,0.010803
4,10168562,20936196,30883841,13,0.010803


In [ ]:
oasis_score.to_csv('oasis_scores.csv')
files.download('oasis_scores.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

query = ''' 

  SELECT 
*
  FROM physionet-data.mimiciv_derived.apsiii
'''
apsiii_score = run_query(query)

apsiii_score.head(5)

,subject_id,hadm_id,stay_id,apsiii,apsiii_prob,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,19295402,29653338,38034263,0,0.011705,0,0,<NA>,0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,18766342,27513687,30509339,0,0.011705,0,0,0,0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0
2,17784939,28330780,32000668,0,0.011705,0,<NA>,0,0,<NA>,...,0,0,<NA>,0,0,<NA>,<NA>,0,<NA>,0
3,17861096,28033870,34455374,0,0.011705,0,0,0,0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0
4,10971284,22087634,33084533,0,0.011705,0,0,0,0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
apsiii_score.to_csv('apsiii_score.csv')
files.download('apsiii_score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

query = ''' 

  SELECT 
stay_id,
starttime,
endtime
  FROM physionet-data.mimiciv_derived.ventilation
  WHERE ventilation_status='InvasiveVent'
'''
lomv = run_query(query)

lomv.head(5)

,stay_id,starttime,endtime
0,31326208,2135-06-20 23:18:00,2135-06-22 01:00:00
1,31195904,2152-08-28 13:00:00,2152-08-29 09:00:00
2,36176896,2115-06-17 15:00:00,2115-06-17 20:00:00
3,35587584,2150-09-27 20:19:00,2150-09-29 09:52:00
4,36179968,2111-05-27 14:07:00,2111-05-27 21:44:00


In [ ]:
lomv.to_csv('lomv.csv')
files.download('lomv.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query='''

SELECT
w.stay_id,
starttime,
weight,
weight_type,

FROM physionet-data.mimiciv_derived.weight_durations w

ORDER BY w.stay_id
'''
hw = run_query(query)
hw.head(5)

,stay_id,starttime,weight,weight_type
0,30000153,2174-09-29 10:09:00,70.0,admit
1,30000153,2174-09-29 16:00:00,73.0,daily
2,30000213,2162-06-21 03:38:00,84.7,admit
3,30000213,2162-06-22 00:00:00,73.7,daily
4,30000484,2136-01-14 15:23:32,68.5,admit


In [ ]:
hw.to_csv('weight.csv')
files.download('weight.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query='''

  SELECT
*

  FROM physionet-data.mimiciv_derived.height

ORDER BY stay_id
'''
height = run_query(query)
height.head(5)

,subject_id,stay_id,charttime,height
0,13180007,30000213,2162-06-21 04:18:00,160.000000000
1,18421337,30000484,2136-01-14 17:23:00,163.000000000
2,12207593,30000646,2194-04-29 01:39:00,170.000000000
3,12980335,30001148,2156-08-30 11:11:00,183.000000000
4,17921898,30002415,2126-12-17 09:54:00,168.000000000


In [ ]:
height.to_csv('height.csv')
files.download('height.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query='''

  SELECT
*

  FROM physionet-data.mimiciv_derived.charlson

ORDER BY subject_id, hadm_id
'''
comorb = run_query(query)
comorb.head(5)

,subject_id,hadm_id,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index
0,10000032,22595853,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,5
1,10000032,22841357,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,3
2,10000032,25742920,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,3
3,10000032,29079034,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,3
4,10000068,25022803,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
comorb.to_csv('comorb.csv')
files.download('comorb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = '''
With icd as
(SELECT
icd_code, long_title

FROM `physionet-data.mimiciv_hosp.d_icd_diagnoses`)

SELECT
di.hadm_id,
icd.long_title
From physionet-data.mimiciv_hosp.diagnoses_icd di
INNER JOIN icd
ON di.icd_code = icd.icd_code

'''

diagnoses = run_query(query)
diagnoses.head(5)

,hadm_id,long_title
0,26381316,"Insomnia, unspecified"
1,25242409,"Nephritis and nephropathy, not specified as acute or chronic, in diseases classified elsewhere"
2,25242409,Atherosclerosis of native arteries of the extremities with intermittent claudication
3,25242409,"Body Mass Index 26.0-26.9, adult"
4,25242409,Nontraumatic hematoma of soft tissue


In [ ]:
diagnoses.to_csv('diagnoses.csv')
files.download('diagnoses.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
WITH diag AS (
    SELECT
        hadm_id
        , CASE WHEN icd_version = 9 THEN icd_code ELSE NULL END AS icd9_code
        , CASE WHEN icd_version = 10 THEN icd_code ELSE NULL END AS icd10_code
    FROM `physionet-data.mimiciv_hosp.diagnoses_icd`
)

, com AS (
    SELECT
        ad.hadm_id

        -- Myocardial infarction
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('410', '412')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I21', 'I22')
            OR
            SUBSTR(icd10_code, 1, 4) = 'I252'
            THEN 1
            ELSE 0 END) AS myocardial_infarct

        -- Congestive heart failure
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '428'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('39891', '40201', '40211', '40291', '40401', '40403'
                , '40411', '40413', '40491', '40493')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4254' AND '4259'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I43', 'I50')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I099', 'I110', 'I130', 'I132', 'I255', 'I420'
                  , 'I425', 'I426', 'I427', 'I428', 'I429', 'P290'
            )
            THEN 1
            ELSE 0 END) AS congestive_heart_failure

        -- Peripheral vascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('440', '441')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0930', '4373', '4471', '5571', '5579', 'V434')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '4431' AND '4439'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('I70', 'I71')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I731', 'I738', 'I739', 'I771', 'I790'
                                         , 'I792'
                                         , 'K551'
                                         , 'K558'
                                         , 'K559'
                                         , 'Z958'
                                         , 'Z959'
            )
            THEN 1
            ELSE 0 END) AS peripheral_vascular_disease

        -- Cerebrovascular disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '430' AND '438'
            OR
            SUBSTR(icd9_code, 1, 5) = '36234'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G45', 'G46')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'I60' AND 'I69'
            OR
            SUBSTR(icd10_code, 1, 4) = 'H340'
            THEN 1
            ELSE 0 END) AS cerebrovascular_disease

        -- Dementia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '290'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('2941', '3312')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('F00', 'F01', 'F02', 'F03', 'G30')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('F051', 'G311')
            THEN 1
            ELSE 0 END) AS dementia

        -- Chronic pulmonary disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '490' AND '505'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4168', '4169', '5064', '5081', '5088')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J40' AND 'J47'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'J60' AND 'J67'
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I278', 'I279', 'J684', 'J701', 'J703')
            THEN 1
            ELSE 0 END) AS chronic_pulmonary_disease

        -- Rheumatic disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) = '725'
            OR
            SUBSTR(icd9_code, 1, 4) IN ('4465', '7100', '7101', '7102', '7103'
                                        , '7104', '7140', '7141', '7142', '7148'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('M05', 'M06', 'M32', 'M33', 'M34')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('M315', 'M351', 'M353', 'M360')
            THEN 1
            ELSE 0 END) AS rheumatic_disease

        -- Peptic ulcer disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('531', '532', '533', '534')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('K25', 'K26', 'K27', 'K28')
            THEN 1
            ELSE 0 END) AS peptic_ulcer_disease

        -- Mild liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('570', '571')
            OR
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('0706', '0709', '5733', '5734', '5738', '5739', 'V427')
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN ('07022', '07023', '07032', '07033', '07044', '07054')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B18', 'K73', 'K74')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('K700', 'K701', 'K702', 'K703', 'K709', 'K713'
                  , 'K714', 'K715', 'K717', 'K760', 'K762'
                  , 'K763', 'K764', 'K768', 'K769', 'Z944')
            THEN 1
            ELSE 0 END) AS mild_liver_disease

        -- Diabetes without chronic complication
        , MAX(CASE WHEN
            SUBSTR(
                icd9_code, 1, 4
            ) IN ('2500', '2501', '2502', '2503', '2508', '2509')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E100', 'E101', 'E106', 'E108', 'E109', 'E110', 'E111'
                  , 'E116'
                  , 'E118'
                  , 'E119'
                  , 'E120'
                  , 'E121'
                  , 'E126'
                  , 'E128'
                  , 'E129'
                  , 'E130'
                  , 'E131'
                  , 'E136'
                  , 'E138'
                  , 'E139'
                  , 'E140'
                  , 'E141', 'E146', 'E148', 'E149')
            THEN 1
            ELSE 0 END) AS diabetes_without_cc

        -- Diabetes with chronic complication
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('2504', '2505', '2506', '2507')
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('E102', 'E103', 'E104', 'E105', 'E107', 'E112', 'E113'
                  , 'E114'
                  , 'E115'
                  , 'E117'
                  , 'E122'
                  , 'E123'
                  , 'E124'
                  , 'E125'
                  , 'E127'
                  , 'E132'
                  , 'E133'
                  , 'E134'
                  , 'E135'
                  , 'E137'
                  , 'E142'
                  , 'E143', 'E144', 'E145', 'E147')
            THEN 1
            ELSE 0 END) AS diabetes_with_cc

        -- Hemiplegia or paraplegia
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('342', '343')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('3341', '3440', '3441', '3442'
                                        , '3443', '3444', '3445', '3446', '3449'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('G81', 'G82')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('G041', 'G114', 'G801', 'G802', 'G830'
                                         , 'G831'
                                         , 'G832'
                                         , 'G833'
                                         , 'G834'
                                         , 'G839'
            )
            THEN 1
            ELSE 0 END) AS paraplegia

        -- Renal disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('582', '585', '586', 'V56')
            OR
            SUBSTR(icd9_code, 1, 4) IN ('5880', 'V420', 'V451')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5830' AND '5837'
            OR
            SUBSTR(
                icd9_code, 1, 5
            ) IN (
                '40301'
                , '40311'
                , '40391'
                , '40402'
                , '40403'
                , '40412'
                , '40413'
                , '40492'
                , '40493'
            )
            OR
            SUBSTR(icd10_code, 1, 3) IN ('N18', 'N19')
            OR
            SUBSTR(icd10_code, 1, 4) IN ('I120', 'I131', 'N032', 'N033', 'N034'
                                         , 'N035'
                                         , 'N036'
                                         , 'N037'
                                         , 'N052'
                                         , 'N053'
                                         , 'N054'
                                         , 'N055'
                                         , 'N056'
                                         , 'N057'
                                         , 'N250'
                                         , 'Z490'
                                         , 'Z491'
                                         , 'Z492'
                                         , 'Z940'
                                         , 'Z992'
            )
            THEN 1
            ELSE 0 END) AS renal_disease

        -- Any malignancy, including lymphoma and leukemia,
        -- except malignant neoplasm of skin.
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) BETWEEN '140' AND '172'
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '1740' AND '1958'
            OR
            SUBSTR(icd9_code, 1, 3) BETWEEN '200' AND '208'
            OR
            SUBSTR(icd9_code, 1, 4) = '2386'
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C43', 'C88')
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C00' AND 'C26'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C30' AND 'C34'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C37' AND 'C41'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C45' AND 'C58'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C60' AND 'C76'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C81' AND 'C85'
            OR
            SUBSTR(icd10_code, 1, 3) BETWEEN 'C90' AND 'C97'
            THEN 1
            ELSE 0 END) AS malignant_cancer

        -- Moderate or severe liver disease
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 4) IN ('4560', '4561', '4562')
            OR
            SUBSTR(icd9_code, 1, 4) BETWEEN '5722' AND '5728'
            OR
            SUBSTR(
                icd10_code, 1, 4
            ) IN ('I850', 'I859', 'I864', 'I982', 'K704', 'K711'
                  , 'K721', 'K729', 'K765', 'K766', 'K767')
            THEN 1
            ELSE 0 END) AS severe_liver_disease

        -- Metastatic solid tumor
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('196', '197', '198', '199')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('C77', 'C78', 'C79', 'C80')
            THEN 1
            ELSE 0 END) AS metastatic_solid_tumor

        -- AIDS/HIV
        , MAX(CASE WHEN
            SUBSTR(icd9_code, 1, 3) IN ('042', '043', '044')
            OR
            SUBSTR(icd10_code, 1, 3) IN ('B20', 'B21', 'B22', 'B24')
            THEN 1
            ELSE 0 END) AS aids
    FROM `physionet-data.mimiciv_hosp.admissions` ad
    LEFT JOIN diag
        ON ad.hadm_id = diag.hadm_id
    GROUP BY ad.hadm_id
)

, ag AS (
    SELECT
        hadm_id
        , age
        , CASE WHEN age <= 50 THEN 0
            WHEN age <= 60 THEN 1
            WHEN age <= 70 THEN 2
            WHEN age <= 80 THEN 3
            ELSE 4 END AS age_score
    FROM `physionet-data.mimiciv_derived.age`
)

SELECT
    ad.subject_id
    , ad.hadm_id
    , ag.age_score
    , myocardial_infarct
    , congestive_heart_failure
    , peripheral_vascular_disease
    , cerebrovascular_disease
    , dementia
    , chronic_pulmonary_disease
    , rheumatic_disease
    , peptic_ulcer_disease
    , mild_liver_disease
    , diabetes_without_cc
    , diabetes_with_cc
    , paraplegia
    , renal_disease
    , malignant_cancer
    , severe_liver_disease
    , metastatic_solid_tumor
    , aids
    -- Calculate the Charlson Comorbidity Score using the original
    -- weights from Charlson, 1987.
    , age_score
    + myocardial_infarct + congestive_heart_failure
    + peripheral_vascular_disease + cerebrovascular_disease
    + dementia + chronic_pulmonary_disease
    + rheumatic_disease + peptic_ulcer_disease
    + GREATEST(mild_liver_disease, 3 * severe_liver_disease)
    + GREATEST(2 * diabetes_with_cc, diabetes_without_cc)
    + GREATEST(2 * malignant_cancer, 6 * metastatic_solid_tumor)
    + 2 * paraplegia + 2 * renal_disease
    + 6 * aids
    AS charlson_comorbidity_index
FROM `physionet-data.mimiciv_hosp.admissions` ad
LEFT JOIN com
    ON ad.hadm_id = com.hadm_id
LEFT JOIN ag
    ON com.hadm_id = ag.hadm_id

'''

comorb = run_query(query)
comorb.head(5)

,subject_id,hadm_id,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index
0,11293517,24643356,2,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,6
1,17187522,24775483,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
2,18126966,28314801,3,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7
3,12513239,21408783,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
4,11653201,21990220,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2


In [ ]:
comorb.to_csv('comorb2.csv')
files.download('comorb2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query= '''
SELECT ie.subject_id, ie.hadm_id, ie.stay_id

    -- patient level factors
    , pat.gender, pat.dod

    -- hospital level factors
    , adm.admittime, adm.dischtime
    , DATETIME_DIFF(adm.dischtime, adm.admittime, DAY) AS los_hospital
    -- calculate the age as anchor_age (60) plus difference between
    -- admit year and the anchor year.
    -- the noqa retains the extra long line so the 
    -- convert to postgres bash script works
    , pat.anchor_age + DATETIME_DIFF(adm.admittime, DATETIME(pat.anchor_year, 1, 1, 0, 0, 0), YEAR) AS admission_age -- noqa: L016
    , adm.race
    , adm.hospital_expire_flag
    , DENSE_RANK() OVER (
        PARTITION BY adm.subject_id ORDER BY adm.admittime
    ) AS hospstay_seq
    , CASE
        WHEN
            DENSE_RANK() OVER (
                PARTITION BY adm.subject_id ORDER BY adm.admittime
            ) = 1 THEN True
        ELSE False END AS first_hosp_stay

    -- icu level factors
    , ie.intime AS icu_intime, ie.outtime AS icu_outtime
    , ROUND(
        CAST(DATETIME_DIFF(ie.outtime, ie.intime, HOUR) / 24.0 AS NUMERIC), 2
    ) AS los_icu
    , DENSE_RANK() OVER (
        PARTITION BY ie.hadm_id ORDER BY ie.intime
    ) AS icustay_seq

    -- first ICU stay *for the current hospitalization*
    , CASE
        WHEN
            DENSE_RANK() OVER (
                PARTITION BY ie.hadm_id ORDER BY ie.intime
            ) = 1 THEN True
        ELSE False END AS first_icu_stay

FROM `physionet-data.mimiciv_icu.icustays` ie
INNER JOIN `physionet-data.mimiciv_hosp.admissions` adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN `physionet-data.mimiciv_hosp.patients` pat
    ON ie.subject_id = pat.subject_id

'''

icu_detail= run_query(query)
icu_detail.head(5)

,subject_id,hadm_id,stay_id,gender,dod,admittime,dischtime,los_hospital,admission_age,race,hospital_expire_flag,hospstay_seq,first_hosp_stay,icu_intime,icu_outtime,los_icu,icustay_seq,first_icu_stay
0,19567799,20023678,38674931,M,NaT,2137-06-10 14:21:00,2137-06-19 14:41:00,9,75,WHITE,0,1,True,2137-06-14 09:48:00,2137-06-15 12:08:42,1.130000000,1,True
1,19141681,20025072,35756582,F,2180-04-18,2178-08-03 12:26:00,2178-08-07 18:20:00,4,55,WHITE - RUSSIAN,0,1,True,2178-08-03 13:22:00,2178-08-05 17:00:53,2.170000000,1,True
2,12605023,20042619,30828113,M,NaT,2196-05-08 21:17:00,2196-05-19 16:03:00,11,74,WHITE,0,1,True,2196-05-12 16:10:56,2196-05-13 21:05:11,1.210000000,1,True
3,12605023,20042619,39552237,M,NaT,2196-05-08 21:17:00,2196-05-19 16:03:00,11,74,WHITE,0,1,True,2196-05-15 19:14:20,2196-05-16 13:26:05,0.750000000,2,False
4,11235565,20051137,30261719,F,NaT,2142-12-26 03:42:00,2142-12-27 16:00:00,1,86,WHITE,0,1,True,2142-12-26 05:07:00,2142-12-27 16:07:52,1.460000000,1,True


In [ ]:
icu_detail.to_csv('icu_detail.csv')
files.download('icu_detail.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

  SELECT 
  stay_id, starttime, sofa_24hours
  FROM physionet-data.mimiciv_derived.sofa

'''
sofa_score = run_query(query)

sofa_score.head(5)

,stay_id,starttime,sofa_24hours
0,37667477,2182-08-06 20:00:00,16
1,32241321,2135-03-11 15:00:00,16
2,31649316,2124-04-29 11:00:00,15
3,38435398,2114-03-01 19:00:00,16
4,33316407,2163-05-04 08:00:00,15


In [ ]:
sofa_score.to_csv('sofa_score.csv')
files.download('sofa_score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

  SELECT 
sirs.stay_id, sirs.sirs
  FROM physionet-data.mimiciv_derived.sirs sirs
'''
sirs_score = run_query(query)

sirs_score.head(5)

,stay_id,sirs
0,39486529,0
1,32520447,0
2,37484422,0
3,35019933,0
4,33084533,0


In [ ]:
sirs_score.to_csv('sirs_score.csv')
files.download('sirs_score.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

  SELECT 
*
  FROM `physionet-data.mimiciv_derived.vasoactive_agent`
'''
vasoactive = run_query(query)

vasoactive.head(5)

,stay_id,starttime,endtime,dopamine,epinephrine,norepinephrine,phenylephrine,vasopressin,dobutamine,milrinone
0,31326208,2135-06-21 03:54:00,2135-06-21 04:01:00,15.569395,NaN,NaN,4.453638,NaN,NaN,NaN
1,31326208,2135-06-21 04:05:00,2135-06-21 04:20:00,NaN,NaN,NaN,4.453638,NaN,NaN,NaN
2,31326208,2135-06-21 06:31:00,2135-06-21 06:57:00,NaN,NaN,0.100075,5.404105,NaN,NaN,NaN
3,31326208,2135-06-21 04:01:00,2135-06-21 04:05:00,9.730874,NaN,NaN,4.453638,NaN,NaN,NaN
4,31326208,2135-06-21 02:01:00,2135-06-21 03:20:00,20.335536,0.05011,NaN,3.996855,NaN,NaN,NaN


In [ ]:
vasoactive.to_csv('vasoactive.csv')
files.download('vasoactive.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
query = ''' 

  SELECT 
*
  FROM `physionet-data.mimiciv_derived.rrt`
'''
rrt = run_query(query)

rrt.head(5)

,stay_id,charttime,dialysis_present,dialysis_active,dialysis_type
0,36066456,2164-01-25 18:00:00,1,0,CVVHD
1,36049628,2185-11-25 09:04:00,1,0,CVVHD
2,36984214,2122-09-11 10:00:00,1,0,Peritoneal
3,39240078,2146-11-07 06:00:00,1,0,CVVHD
4,32172367,2168-04-18 18:00:00,1,0,Peritoneal


In [ ]:
rrt.to_csv('rrt.csv')
files.download('rrt.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>